In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
import pandas as pd
import bs4
import requests

In [3]:
url = 'https://news.daum.net'
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html, 'html.parser')

In [4]:
ul = bs_obj.find('ul', {'class':'gnb_comm'})

In [5]:
# 수집된 데이터로 df 구성

lis = ul.select('li')

section = []
link = []

# 리스트에 추가 
for li in lis:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append(url+a_tag['href'])

section_menu_df = pd.DataFrame({'section':section, 'link':link})
#section_menu_df

In [6]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
res = requests.get('https://news.daum.net/society', headers=headers)
html = res.text
bs_obj = bs4.BeautifulSoup(html, 'html.parser')
#bs_obj

In [7]:
# 해당 섹션에 대한 모든 topic 크롤링하는 함수 생성
def get_topic(url, section):
    # 접속 및 결과 파싱
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser') 
    
    # topic_list 추출
    ul = bs_obj.find('ul', {'class':'list_mainnews'})
    topic_list = ul.select('li')

    # 제목과 링크 추출해서 리스트에 추가
    topic_title = []
    topic_link = []

    for topic in topic_list:    
        try :
            a = topic.find('a')
            topic_title.append(topic.select_one('strong').text.strip())
            topic_link.append(topic.select_one('a')['href'])
        except:
            print('오류 발생')
            # pass

    # 딕셔너리 만들어서 반환
    return ({'topic':topic_title, 'url':topic_link, 'section':section})

In [8]:
# 모든 섹션에 대한 모든 topic 데이터프레임 생성
all_topic_df = pd.DataFrame({
    'topic':[],
    'url': [],
    'section': []
})

for i in range(1, 7) :
    df = pd.DataFrame(get_topic(section_menu_df['link'][i],  section_menu_df['section'][i]))
    all_topic_df = pd.concat([all_topic_df, df], axis=0, ignore_index=True)

#all_topic_df

In [9]:
# sub 페이지 기사 내용 추출하는 함수
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    # 빈 리스트 생성
    paper_list, title_list, datetime_list, writing_list, writer_list = [ [] for _ in range(5) ]
    # for _ in range(5) : 언더바(_) (변수 없이 반복만 수행)
    
    try :
        
        res = requests.get(url, headers=headers)
        html = res.text
        bs_obj = bs4.BeautifulSoup(html, 'html.parser')
        # 접속    
    
        # 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자 리스트 생성
        paper_list.append(bs_obj.select_one('#kakaoServiceLogo').text)
        title_list.append(bs_obj.find('h3', {'class' : 'tit_view'}).text)
        datetime_list.append(bs_obj.find('span', {'class' : 'num_date'}).text)
        writing_list.append(bs_obj.find('div', {'class' : 'article_view'}).text.strip().replace('\n', ''))
        writer_list.append(bs_obj.find('span',{'class':'txt_info'}).text)
        #writer_list.append(bs_obj.select('section section p')[-1].text)
    except:
        print("오류 발생")
        # pass


    # 딕셔너리로 만들어서 반환
    sub_news_dict = {
        'paper':paper_list,
        'title':title_list,
        'datetime':datetime_list,
        'writing':writing_list,
        'writer':writer_list
    }

    return sub_news_dict

In [10]:
#get_sub_news_info('https://v.daum.net/v/20230913143033928')

In [11]:
# 모든 sub 페이지 내용 저장하는 데이터프레임 생성해서 최종 데이터프레임 생성
all_sub_news_info_df = pd.DataFrame({
    'paper': [],
    'title': [],
    'datetime': [],
    'writing': [],
    'writer': []
})

for url in all_topic_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)

#all_sub_news_info_df

In [12]:
# 최종 결과 파일로 저장
all_sub_news_info_df.to_csv('./crawl_data/naver_sub_news_section.csv', index=0)

In [13]:
all_sub_news_info_df = pd.read_csv('./crawl_data/naver_sub_news_section.csv')
all_sub_news_info_df

,paper,title,datetime,writing,writer
0,뉴시스,추석연휴 인천공항서 110만명 '해외로'...코로나 이전 90% 운항률,2023. 9. 13. 15:00,[인천공항=뉴시스] 정병혁 기자 = 10월 2일이 임시공휴일로 지정되면서 추석 연휴...,홍찬선 기자
1,더팩트,고교 때 훈계했다고…스승 찾아가 난동 20대 체포,2023. 9. 13. 14:57,13일 경찰에 따르면 서울 강동경찰서는 전날 20대 남성 A씨를 건조물침입 혐의로 ...,조소현
2,연합뉴스,"음주운전으로 강등됐던 해경, 다시 음주운전 했다가 파면",2023. 9. 13. 15:02,해양경찰청 [해양경찰청 제공. 재판매 및 DB 금지] (부산=연합...,차근호
3,연합뉴스,"방심위, 디지털성범죄 유통 근절 위한 국제협력 강화",2023. 9. 13. 15:01,방송통신심의위원회 [방송통신심의위원회 페이스북 캡처] (서울=연합...,이정현
4,YTN,[자막뉴스] 수당 부정 수령 '수두룩'... 공무원의 수상한 행적,2023. 9. 13. 15:01,날이 일찍 저문 저녁 한 무리가 건물 안으로 들어갑니다. 이들이 찾은 곳은 골프연습...,YTN
5,연합뉴스,"20년 넘은 인천공항 제1터미널, 1조 투입해 시설 전반 개선",2023. 9. 13. 15:00,출국 준비하는 공항 이용객들 [연합뉴스 자료사진] (영종도=연합뉴...,김잔디
6,연합뉴스,"대구시, 9월 재산세 4천22억원 부과…작년 대비 8.5% 감소",2023. 9. 13. 15:03,[연합뉴스 자료사진] (대구=연합뉴스) 이덕기 기자 = 대구지역 ...,이덕기
7,경향신문,"역사학계 “정부 왜곡으로 평민 의병장 홍범도, 부관참시당해”",2023. 9. 13. 15:03,한국역사연구회 등 역사단체 회원들이 13일 서울대학교에서 ‘홍범도 장군 흉상 철거로...,전지현 기자
8,프레시안,"방문규 인사청문회…부모 소득공제, 자녀 유학 위법 논란",2023. 9. 13. 15:01,[곽재훈 기자(nowhere@pressian.com)]방문규 산업통상자원부 장관 후...,입력 2023. 9. 13. 15:01
9,파이낸셜뉴스,北 김정은-푸틴회담 앞두고 미사일 깜짝 도발..군사분야 협상력 높이려는 세리머니?,2023. 9. 13. 14:48,[파이낸셜뉴스] 북한 김정은이 지난 3월 ...,이종윤
